https://github.com/oziieljuniior/Leitor_PDF

# Primeira parte

Essa parte é foca em carregar o pdf e organizar os dados do pdf em uma pasta source para o modelo.

In [2]:
import os
import fitz  # PyMuPDF
import faiss
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import textwrap


/home/darkcover/.cache/pypoetry/virtualenvs/leitor-pdf-S8aY7ORV-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Parâmetros
CHUNK_SIZE = 300  # caracteres por chunk
PDF_PATH = "/home/darkcover/Documentos/Leitor_PDF/documentos/SAGE_ManCfg_Anx17_61850 1.pdf"
OUTPUT_DIR = "partes_pdf"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Modelo de embeddings leve
embed_model = SentenceTransformer('paraphrase-albert-small-v2')

In [4]:

# Função para dividir PDF em partes menores (ex: 10 páginas cada)
def dividir_pdf_em_partes(pdf_path, paginas_por_parte=10):
    doc = fitz.open(pdf_path)
    num_paginas = doc.page_count
    partes = []

    for i in range(0, num_paginas, paginas_por_parte):
        subdoc = fitz.open()
        for j in range(i, min(i + paginas_por_parte, num_paginas)):
            subdoc.insert_pdf(doc, from_page=j, to_page=j)
        parte_path = os.path.join(OUTPUT_DIR, f"parte_{i // paginas_por_parte + 1}.pdf")
        subdoc.save(parte_path)
        partes.append(parte_path)

    return partes

# Extrair texto do PDF
def extrair_texto(pdf_path):
    doc = fitz.open(pdf_path)
    texto = ""
    for pagina in doc:
        texto += pagina.get_text()
    return texto

# Quebrar texto em chunks menores
def chunk_text(text, max_chars=CHUNK_SIZE):
    return textwrap.wrap(text, width=max_chars)

# Salvar FAISS e chunks
def salvar_index_e_chunks(chunks, nome_base):
    embeddings = embed_model.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))

    faiss.write_index(index, f"{nome_base}.faiss")
    with open(f"{nome_base}_chunks.pkl", "wb") as f:
        pickle.dump(chunks, f)

# Pipeline principal de processamento
def processar_partes(pdf_path):
    partes = dividir_pdf_em_partes(pdf_path)
    for parte in partes:
        texto = extrair_texto(parte)
        chunks = chunk_text(texto)
        base = os.path.splitext(parte)[0]
        salvar_index_e_chunks(chunks, base)


In [7]:
# Rodar o pipeline
processar_partes(PDF_PATH)

# Mostrar os arquivos gerados
import os
import pandas as pd

arquivos_gerados = os.listdir(OUTPUT_DIR)
df = pd.DataFrame(arquivos_gerados, columns=["Arquivo"])
print(df)  # ou use display(df) se estiver no Jupyter Notebook


               Arquivo
0   parte_6_chunks.pkl
1        parte_3.faiss
2          parte_5.pdf
3   parte_2_chunks.pkl
4        parte_1.faiss
5        parte_4.faiss
6        parte_6.faiss
7   parte_4_chunks.pkl
8          parte_2.pdf
9          parte_6.pdf
10  parte_5_chunks.pkl
11       parte_2.faiss
12         parte_4.pdf
13         parte_1.pdf
14       parte_5.faiss
15  parte_3_chunks.pkl
16         parte_3.pdf
17  parte_1_chunks.pkl


---

# Segunda Parte

In [9]:
import os
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama


In [12]:

# Parâmetros
PARTES_DIR = "partes_pdf"
TOP_K = 2  # quantidade de chunks por parte
MAX_CONTEXT = 5  # máximo de chunks para resposta
MODEL_PATH = "/home/darkcover/Documentos/Leitor_PDF/models/phi-2.Q4_K_M.gguf"

# Carrega modelo de embeddings
embed_model = SentenceTransformer("paraphrase-albert-small-v2")

# Carrega modelo LLM local
llm = Llama(model_path=MODEL_PATH, n_ctx=2048, n_threads=4)

# Função para buscar chunks relevantes em um par de arquivos .faiss + .pkl
def buscar_chunks_relevantes(faiss_path, pkl_path, pergunta):
    with open(pkl_path, "rb") as f:
        chunks = pickle.load(f)

    index = faiss.read_index(faiss_path)
    pergunta_emb = embed_model.encode([pergunta])
    D, I = index.search(np.array(pergunta_emb), TOP_K)
    return [chunks[i] for i in I[0]]

# Junta todos os contextos das partes
def montar_contexto_global(pergunta):
    contextos = []
    arquivos = os.listdir(PARTES_DIR)
    partes = sorted(set(f.split(".")[0] for f in arquivos if f.endswith(".faiss")))

    for parte in partes:
        faiss_file = os.path.join(PARTES_DIR, f"{parte}.faiss")
        chunk_file = os.path.join(PARTES_DIR, f"{parte}_chunks.pkl")
        if os.path.exists(faiss_file) and os.path.exists(chunk_file):
            contextos.extend(buscar_chunks_relevantes(faiss_file, chunk_file, pergunta))

    return "\n".join(contextos[:MAX_CONTEXT])

# Gera resposta com modelo local
def gerar_resposta(pergunta):
    contexto = montar_contexto_global(pergunta)
    prompt = f"""Você é um assistente inteligente. Use o contexto abaixo para responder a pergunta.

Contexto:
{contexto}

Pergunta:
{pergunta}

Resposta:"""
    resposta = llm(prompt, max_tokens=200)
    return resposta["choices"][0]["text"].strip()


llama_model_loader: loaded meta data with 20 key-value pairs and 325 tensors from /home/darkcover/Documentos/Leitor_PDF/models/phi-2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi2.atten

In [13]:

# Exemplo de uso
if __name__ == "__main__":
    pergunta = input("Digite sua pergunta sobre o PDF: ")
    resposta = gerar_resposta(pergunta)
    print("\n🧠 Resposta:")
    print(resposta)


llama_perf_context_print:        load time =   54247.26 ms
llama_perf_context_print: prompt eval time =   54239.76 ms /   534 tokens (  101.57 ms per token,     9.85 tokens per second)
llama_perf_context_print:        eval time =   96141.26 ms /   199 runs   (  483.12 ms per token,     2.07 tokens per second)
llama_perf_context_print:       total time =  150983.34 ms /   733 tokens



🧠 Resposta:
Os instrumentos que são utilizados na Subestação SB  – SBa  ■ Logical Devices do IED    – Control e Measurement  ■ Chave Seccionadora da Subestação – QA1  ■ Instância de XSWI no IED    – 
2  ■ Transformador da Subestação  – TF5  ■ Instância de MMXU no IED    – 
3    Uma lista com os principais Logical Nodes definidos pela norma no
configurados na base de dados do SAGE e  que existem no IED.  Caso contrário, o SAGE usará os Data Sets pré-existentes no IED e criará  17 CONFIGURAÇÃO PARA COMUNICAÇÃO COM IEDS EM PROTOCOLO I
